## 機械学習サンプル

* GAT2017のログを100戦ぶんだけ読み込む(ほんとは10万戦あります)  
* 各試合の1-5日目の投票前及び2-6日目の朝の時点までのログを抽出(1試合あたり10セット)  
* Tensor5460を使って、5460パターンにサンプルエージェントが使っている特徴量を付与  
* 人狼と狂人の真の位置を使って0/1フラグ付与(1/5460だけ1)  
* 全部縦に積んで(100x10x5460行)ロジスティック回帰    

In [11]:
from __future__ import print_function, division 

import os 
import sys

import numpy as np
import pandas as pd
import sklearn.linear_model
import math

In [2]:
# add sys.path for aiwolfpy
sys.path.append("../AIWolfPy")

In [3]:
import aiwolfpy
import aiwolfpy.cash

In [4]:
predictor = aiwolfpy.cash.Predictor_15()

In [5]:
def game_data_filter(df, day, phase='daily_initialize', agent=0):
    
    
    y = np.zeros(5460)
    # werewolves, possessed
    werewolves = []
    for i in range(1, 16):
        role = df["text"][i - 1].split()[2]
        if role == "WEREWOLF":
            werewolves.append(i)
        elif role == "POSSESSED":
            possessed = i
            
    for i in range(5460):
        if predictor.case15.case5460_df["agent_"+str(possessed)][i] == 2:
            if predictor.case15.case5460_df["agent_"+str(werewolves[0])][i] == 1:
                if predictor.case15.case5460_df["agent_"+str(werewolves[1])][i] == 1:
                    if predictor.case15.case5460_df["agent_"+str(werewolves[2])][i] == 1:
                        y[i] = 1
    
    # role
    role = "VILLAGER"
    if agent > 0:
        role = df["text"][agent - 1].split()[2]
    
    # filter by role
    if role in ["VILLAGER", "POSSESSED"]:
        df = df[df["type"].isin(["talk", "vote", "execute", "dead"])]
    elif role == "MEDIUM":
        df = df[df["type"].isin(["talk", "vote", "execute", "dead", "identify"])]
    elif role == "SEER":
        df = df[df["type"].isin(["talk", "vote", "execute", "dead", "divine"])]
    elif role == "BODYGUARD":
        df = df[df["type"].isin(["talk", "vote", "execute", "dead", "guard"])]
    elif role == "WEREWOLF":
        df = df[df["type"].isin(["talk", "vote", "execute", "dead", "whisper", "attack", "attack_vote"])]
    
    
    # agent
    if agent == 0:
        agent = 1
    
    # filter by time
    if phase == 'daily_initialize':
        df = df[df["day"] < day]
    else:
        df = df[(df["day"] < day) | ((df["day"] == day) & (df["type"] == 'talk'))]
    
    predictor.initialize({"agent":agent, "roleMap":{str(agent):role}}, {})
    predictor.update_features(df.reset_index())
    predictor.update_df()
    
    return predictor.df_pred, y

In [6]:
log_path = "../log/gat2017log15/000/000.log"

In [7]:
x, y = game_data_filter(aiwolfpy.read_log(log_path), day=3, phase='vote')

In [9]:
# build data for 100 games
# takes several minutes
x_1000 = np.zeros((5460000, 60))
y_1000 = np.zeros(5460000)

ind = 0
for i in range(100):
    log_path = "../log/gat2017log15/000/" + "{0:03d}".format(i) + ".log"
    for d in range(5):
        x, y = game_data_filter(aiwolfpy.read_log(log_path), day=d, phase='vote')
        x_1000[(ind*5460):((ind+1)*5460), :] = x
        y_1000[(ind*5460):((ind+1)*5460)] = y
        ind += 1
        x, y = game_data_filter(aiwolfpy.read_log(log_path), day=d+1, phase='daily_initialize')
        x_1000[(ind*5460):((ind+1)*5460), :] = x
        y_1000[(ind*5460):((ind+1)*5460)] = y
        ind += 1

In [12]:
model = sklearn.linear_model.LogisticRegression()
model.fit(x_1000, y_1000)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
np.set_printoptions(suppress=True)

In [15]:
model.coef_

array([[-0.05142977,  0.00683707, -4.21711505,  0.4640681 , -3.13746134,
         0.07285381, -0.58640055,  0.70195596, -3.11076185,  0.88402891,
         0.0773789 ,  0.50954873, -2.80776988,  0.3146642 , -1.5538348 ,
         0.12254767,  0.05153261,  0.27717954, -0.2773668 ,  0.14004431,
        -0.63454665,  0.23758233, -2.5502184 ,  0.16825324, -2.36900027,
         0.19680779,  0.00372291, -2.57349345, -0.73518057,  0.07310151,
        -0.1628885 , -0.04290204,  0.53040538, -0.39452875, -0.18142595,
        -0.0251955 ,  0.07128287,  0.28275352, -0.20261612, -0.140135  ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.02784234,  0.32486888, -0.53507881, -0.49403682, -0.87516125,
        -0.22883512, -4.20619298, -0.01235312,  1.09991243,  0.9656281 ,
        -0.3238915 ,  0.02788588,  2.77381813, -1.50430048, -0.08328738]])

In [13]:
x.columns

Index(['VOTE_HH', 'DIV_HM_HH', 'DIV_WW_HH', 'IDT_HM_HH', 'IDT_WW_HH',
       'VOTE_HW', 'DIV_HM_HW', 'DIV_WW_HW', 'IDT_HM_HW', 'IDT_WW_HW',
       'VOTE_HP', 'DIV_HM_HP', 'DIV_WW_HP', 'IDT_HM_HP', 'IDT_WW_HP',
       'VOTE_WH', 'DIV_HM_WH', 'DIV_WW_WH', 'IDT_HM_WH', 'IDT_WW_WH',
       'VOTE_WW', 'DIV_HM_WW', 'DIV_WW_WW', 'IDT_HM_WW', 'IDT_WW_WW',
       'VOTE_WP', 'DIV_HM_WP', 'DIV_WW_WP', 'IDT_HM_WP', 'IDT_WW_WP',
       'VOTE_PH', 'DIV_HM_PH', 'DIV_WW_PH', 'IDT_HM_PH', 'IDT_WW_PH',
       'VOTE_PW', 'DIV_HM_PW', 'DIV_WW_PW', 'IDT_HM_PW', 'IDT_WW_PW',
       'VOTE_PP', 'DIV_HM_PP', 'DIV_WW_PP', 'IDT_HM_PP', 'IDT_WW_PP',
       'executed_H', 'attacked_H', 'CO_SEER_H', 'CO_MEDIUM_H',
       'CO_BODYGUARD_H', 'executed_W', 'attacked_W', 'CO_SEER_W',
       'CO_MEDIUM_W', 'CO_BODYGUARD_W', 'executed_P', 'attacked_P',
       'CO_SEER_P', 'CO_MEDIUM_P', 'CO_BODYGUARD_P'],
      dtype='object')